In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Your_Path')

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

# Pull Ollama

In [ ]:
!ollama pull gpt-oss:20b

In [ ]:
!pip install langchain-ollama

In [ ]:
import json
from pathlib import Path

# Drive’daki JSON dosyanızın yolu
JSON_PATH = Path("/content/drive/MyDrive/json_path")

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# Text ve metadata listeleri
texts, metadatas = [], []


# Depends on your json data structure
for rec in data:
    soru = rec.get("input", "")
    out  = rec.get("output", {}) or {}
    cevap = out.get("answer", "")
    konu  = out.get("Konu Başlığı", "")
    fak   = out.get("Fakülte", "")

    text = (
        f"Soru: {soru}\n"
        f"Cevap: {cevap}\n"
        f"Konu Başlığı: {konu}\n"
        f"Fakülte: {fak}"
    )
    texts.append(text)
    metadatas.append({"Konu Başlığı": konu, "Fakülte": fak, "source": JSON_PATH.name})

len(texts), texts[0][:250]


# Embedding and Vector DB

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# # Kalıcı indeks klasörü (Drive’da dursun ki tekrar oluşturmak zorunda kalmayalım)
PERSIST_DIR = "/content/drive/MyDrive/Chroma_Rag_Index_Path"

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Eğer ilk kez oluşturuyorsan:
vectordb = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    persist_directory=PERSIST_DIR,
)
vectordb.persist()

# Sonraki oturumlarda mevcut indeksi şu şekilde yükleyebilirsin:
vectordb = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)

print("Vektör sayısı:", vectordb._collection.count())


# Initial Prompt, Formatting and

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

retriever = vectordb.as_retriever(search_kwargs={"k": 4})

# llm = OllamaLLM(model="gpt-oss:20b")

PROMPT = ChatPromptTemplate.from_template("""Aşağıdaki bağlamı kullanarak soruya **Türkçe**, **kısa ve doğru** bir yanıt ver.
Eğer bağlam soruyu yanıtlamak için yetersizse: "Bu konuda elimde veri yok." de.

[BAĞLAM BAŞLANGIÇ]
{context}
[BAĞLAM BİTİŞ]

Soru: {question}
Cevap:""")

def format_docs(docs):
    # Chroma->LangChain dökümanlarını tek metne birleştir
    return "\n\n".join([d.page_content for d in docs])

def ask(question: str):
    docs = retriever.get_relevant_documents(question)
    context = format_docs(docs)
    prompt = PROMPT.format(context=context, question=question)
    return llm.invoke(prompt), docs

# Ask to LLM

In [ ]:
Deneme:
answer, used_docs = ask("YKS dil puanım ile hangi fırsatlara sahip oluyorum üniversitede?")
print(answer)

# RANK_BM_25

In [ ]:
!pip install rank_bm25

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document

# Chroma'ya verdiğin aynı içeriklerden BM25 dokümanları oluştur
bm25_docs = [Document(page_content=t, metadata=m) for t, m in zip(texts, metadatas)]
bm25 = BM25Retriever.from_documents(bm25_docs)
bm25.k = 8  # kaç sonuç istiyorsun

In [ ]:
from langchain_community.vectorstores import Chroma

vectordb = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)

chroma_ret = vectordb.as_retriever(
    search_type="mmr",            # çeşitlilik + alaka
    search_kwargs={"k": 8, "fetch_k": 30}
)

In [ ]:
from langchain.retrievers import EnsembleRetriever

retriever = EnsembleRetriever(
    retrievers=[chroma_ret, bm25],
    weights=[0.6, 0.4]  # vektör biraz daha ağır basın
)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate # Import ChatPromptTemplate

llm_rewriter = OllamaLLM(model="gpt-oss:20b")

MQ_PROMPT = """Aşağıdaki kullanıcı sorusu için Türkçe 4 farklı arama varyantı üret.
Varyantlar kısa, spesifik ve eşanlamlı sorgular olsun; nokta ile bitir.
Soru: {question}
"""

multi_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm_rewriter,
    prompt=ChatPromptTemplate.from_template(MQ_PROMPT) # Convert string to ChatPromptTemplate
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# llm = OllamaLLM(model="gpt-oss:20b")

PROMPT = ChatPromptTemplate.from_template("""Aşağıdaki bağlamı kullanarak soruya **Türkçe**, **kısa ve doğru** bir yanıt ver.
Eğer bağlam soruyu yanıtlamak için yetersizse: "Bu konuda elimde veri yok." de.

[BAĞLAM BAŞLANGIÇ]
{context}
[BAĞLAM BİTİŞ]

Soru: {question}
Cevap:""")

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

def ask(question: str):
    docs = multi_retriever.get_relevant_documents(question)
    context = format_docs(docs)
    prompt = PROMPT.format(context=context, question=question)
    return llm.invoke(prompt), docs

# Örnek: Soyut soru
# ans, used = ask("Özel Eğitim mezunları nerelerde çalışabilir?")
# print(ans)

In [ ]:
ans, used = ask("Ben bir öğrenciyim. Kopya çekilmesi konusunda endişe yaşıyorum. Sağlık Bilimleri Fakültesinde öğrenciyim. Bir öğrencinin kopya çektiği tespit edilince nasıl bir sonuç oluyor? Nasıl Cezalar verilir?")
print(ans)